# Importing Data Example

In order to import data into `pypillometry`, we have to load the data from the source using other packages and then wrap it into `PupilData` objects. 

Here, we will use `pandas.read_csv()`.

First, we import the needed modules.

In [1]:
import sys
sys.path.insert(0,"..") # this is not needed if you have installed pypillometry
import pypillometry as pp
import pandas as pd

In this example, we use data recorded with an Eyelink-eyetracker. These eyetrackers store the files in binary files with the extension `.edf`. Some information about this file-format is [here](http://download.sr-support.com/dispdoc/page25.html). We use a command-line utility released by Eyelink to convert this proprietory format into a more easily read `.asc` file that is a whitespace-separated plain-text format. The converter, `edf2asc` is a program that can be downloaded for different platforms from the [Eyelink support forum](https://www.sr-research.com/support/). There is a GUI-based program for windows and command-line programs for linux and mac. Binaries of the command-line tools for linux and mac are included in `pypillometry` under [this link]().

In [13]:
!../external/edf2asc-linux -y -s ../data/test.edf ../data/test_samples.asc
!../external/edf2asc-linux -y -e ../data/test.edf ../data/test_events.asc


EDF2ASC: EyeLink EDF file -> ASCII (text) file translator
EDF2ASC version 3.0 Linux Dec  1 2008 
(c)1995-2007 by SR Research, last modified Dec  1 2008

processing file ../data/test.edf 
=======================Preamble of file ../data/test.edf=======================
| DATE: Fri Feb 14 08:48:33 2020                                              |
| TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED                                   |
| VERSION: EYELINK II 1                                                       |
| SOURCE: EYELINK CL                                                          |
| EYELINK II CL v6.12 Feb  1 2018 (EyeLink Portable Duo)                      |
| CAMERA: EyeLink USBCAM Version 1.01                                         |
| SERIAL NUMBER: CLU-DAC49                                                    |
| CAMERA_CONFIG: DAC49200.SCD                                                 |
| Psychopy GC demo                                                            |

Converted s

In [8]:
pyedflib.edfreader.EdfReader("../data/1_pav.EDF")

OSError: the file is not EDF(+) or BDF(+) compliant (it contains format errors)

In [16]:

import requests
from io import BytesIO

In [19]:
fstr=BytesIO(res.content)

In [20]:
import pickle

In [22]:
pickle.loads(res.content)

PupilData(55910_1_wu):
 n                 : 12501
 nmiss             : 2291
 nevents           : 10
 nblinks           : 0
 ninterpolated     : 0
 blinks_per_min    : 0.0
 fs                : 500.0
 duration_minutes  : 0.4167
 start_min         : 2.0833333333333335
 end_min           : 2.5
 baseline_estimated: False
 response_estimated: False

In [21]:
pickle.load(fstr)

PupilData(55910_1_wu):
 n                 : 12501
 nmiss             : 2291
 nevents           : 10
 nblinks           : 0
 ninterpolated     : 0
 blinks_per_min    : 0.0
 fs                : 500.0
 duration_minutes  : 0.4167
 start_min         : 2.0833333333333335
 end_min           : 2.5
 baseline_estimated: False
 response_estimated: False

In [26]:
durl="https://github.com/ihrke/pypillometry/raw/master/data/example_shortblinks.pd"
res=requests.get(durl)


In [27]:
res.status_code

200